# Exemples "SPARQL in 11 Minutes"

Ces exemples s'appuient sur un jeu de données utilisant le vocabulaire vocabulaire [VCARD](https://www.w3.org/TR/vcard-rdf/) et s'appuient sur l'exemple évoqué dans la vidéo disponible à l'adresse: <https://www.youtube.com/watch?v=FvGndkpa4K0>

Il interroge le graphe suivant:

```turtle
@prefix vcard: <http://www.w3.org/2006/vcard/ns#> .
@prefix sn: <http://www.snee.com/hr/> .

sn:emp1   vcard:given-name   "Heidi" .
sn:emp1   vcard:family-name   "Smith" .
sn:emp1   vcard:title   "CEO" .
sn:emp1   sn:hireDate   "2015-01-13" .
sn:emp1   sn:completedOrientation   "2015-01-30" .

sn:emp2   vcard:given-name   "John" .
sn:emp2   vcard:family-name   "Smith" .
sn:emp2   sn:hireDate   "2015-01-28" .
sn:emp2   vcard:title   "Engineer" .
sn:emp2   sn:completedOrientation   "2015-01-30" .
sn:emp2   sn:completedOrientation   "2015-03-15" .

sn:emp3   vcard:given-name   "Francis" .
sn:emp3   vcard:family-name   "Jones" .
sn:emp3   sn:hireDate   "2015-02-13" .
sn:emp3   vcard:title   "Vice President" .

sn:emp4   vcard:given-name   "Jane" .
sn:emp4   vcard:family-name   "Berger" .
sn:emp4   sn:hireDate   "2015-03-10" .
sn:emp4   vcard:title   "Sales" .
```

## Configuration du point d'accès

Cette configuration est requise par le noyau de `jupyter` pour atteindre le point d'accès SPARQL

In [3]:
%endpoint http://sparql:3030/sparqlIn11Minutes/sparql
%format any

# Paramétrage du point d'accès

Endpoint set to: http://sparql:3030/sparqlIn11Minutes/sparql
Return format: ANY

## Requête simple

Liste des employés dont le nom de famille est `Smith`

SPARQL utilise des prefixes pour ne pas avoir de longues _URI_

Pour la pluspart des requêtes _SPARQL_, il vaut mieux regarder la clause `WHERE` d'abord, parcequ'elle décrit les triplets qu'on veut retirer du jeu de données: elle exprime les sujets, prédicats et objects qui sont respectivement en adéquation. Les variables commencent par le caractère `?` (ou `$` ) et sont substituées dans le résultat.

La clause `SELECT` décrit les variables qu'on veut sélectionner.

In [4]:
# 3m04s

PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 

SELECT ?person 
WHERE  
{ 
   ?person vcard:family-name "Smith" . 
}

person
http://www.snee.com/hr/emp1
http://www.snee.com/hr/emp2


In [5]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 

SELECT ?person 
WHERE  
{ 
   ?person vcard:title "Vice President" . 
}

person
http://www.snee.com/hr/emp3


## Nom / Prénom

Récupérer le prénom associé au nom.
Les triplets de la clausse `WHERE` doivent correspondre concomitamment (ligne 1 `ET` ligne 2):
La valeur de la variable `?person` de la première ligne de la clause est mémorisée et substituée comme valeur de la deuxième occurence de `?person`.

In [6]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 

SELECT ?person ?givenName
WHERE 
{ 
   ?person vcard:family-name "Smith" . 
   ?person vcard:given-name ?givenName .
}

person,givenName
http://www.snee.com/hr/emp1,Heidi
http://www.snee.com/hr/emp2,John


## Réduction des résultats

### Sélection de la date d'embauche

In [7]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 
PREFIX sn: <http://www.snee.com/hr/> 

SELECT ?givenName ?familyName ?hireDate
WHERE 
{ 
   ?person vcard:given-name ?givenName .
   ?person vcard:family-name ?familyName . 
   ?person sn:hireDate ?hireDate .
}

givenName,familyName,hireDate
Jane,Berger,2015-03-10
Heidi,Smith,2015-01-13
John,Smith,2015-01-28
Francis,Jones,2015-02-13


### Filtrage des résultats

Sélection des enregistrements telles que la date d'embauche est antérieure au 1er Mars 2015.

In [8]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 
PREFIX sn: <http://www.snee.com/hr/> 

SELECT ?givenName ?familyName ?hireDate
WHERE 
{ 
   ?person vcard:given-name ?givenName .
   ?person vcard:family-name ?familyName . 
   ?person sn:hireDate ?hireDate . 
   FILTER(?hireDate < "2015-03-01")
}

givenName,familyName,hireDate
Heidi,Smith,2015-01-13
John,Smith,2015-01-28
Francis,Jones,2015-02-13


## Critère supplémentaires

Il faut que l'ensemble des triplets correspondent pour qu'ils fassent parti des résultats 

In [9]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 
PREFIX sn: <http://www.snee.com/hr/> 

SELECT ?givenName ?familyName ?oDate
WHERE 
{ 
   ?person vcard:given-name ?givenName .
   ?person vcard:family-name ?familyName . 
   ?person sn:completedOrientation  ?oDate . 
}

givenName,familyName,oDate
Heidi,Smith,2015-01-30
John,Smith,2015-01-30
John,Smith,2015-03-15


Pour qu'une clause soit optionnelles, il faut le spécifier avec `OPTIONAL`

In [11]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 
PREFIX sn: <http://www.snee.com/hr/> 

SELECT ?givenName ?familyName ?oDate
WHERE 
{ 
   ?person vcard:given-name ?givenName .
   ?person vcard:family-name ?familyName . 
   OPTIONAL { ?person sn:completedOrientation  ?oDate . }
}

givenName,familyName,oDate
Jane,Berger,
Heidi,Smith,2015-01-30
John,Smith,2015-01-30
John,Smith,2015-03-15
Francis,Jones,


## Négation

Sélection d'un ensemble de variables pour lesquelles une clause n'est pas vérifiée: `NOT EXISTS`

In [12]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 
PREFIX sn: <http://www.snee.com/hr/> 

SELECT ?givenName ?familyName
WHERE 
{ 
   ?person vcard:given-name ?givenName .
   ?person vcard:family-name ?familyName . 
   NOT EXISTS { ?person sn:completedOrientation  ?oDate . }
}

givenName,familyName
Jane,Berger
Francis,Jones


## Affectation de variables

Affectation d'une variable avec une valeur: `BIND(… AS ?…)`

In [13]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX sn: <http://www.snee.com/hr/>

SELECT ?givenName ?familyName ?someVariable WHERE {
  ?person vcard:given-name ?givenName ;
          vcard:family-name ?familyName .
  BIND("some value" AS ?someVariable)
}

givenName,familyName,someVariable
Jane,Berger,some value
Heidi,Smith,some value
John,Smith,some value
Francis,Jones,some value


La variable peut être calculée 

> ici avec l'opération de
concaténation `CONCAT`

In [14]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 
PREFIX sn: <http://www.snee.com/hr/> 

SELECT ?givenName ?familyName ?fullName
WHERE { 
  ?person vcard:given-name ?givenName .  
  ?person vcard:family-name ?familyName .
  BIND(concat(?givenName," ",?familyName) AS ?fullName) 
}

givenName,familyName,fullName
Jane,Berger,Jane Berger
Heidi,Smith,Heidi Smith
John,Smith,John Smith
Francis,Jones,Francis Jones


## Construction de triplets

L'opérateur `CONSTRUCT` permet de fournir un résultat sous forme de graphe: la valeur de retour est un graph construit à partir des triplet définis par l'opérateur.

> Ici on produit un équivalent dans le vocabulaire [FOAF](http://xmlns.com/foaf/spec/) des information spécifiées avec le vocabulaire [VCARD](https://www.w3.org/TR/vcard-rdf/)

In [15]:
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX sn: <http://www.snee.com/hr/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
  ?person rdf:type foaf:Person .
  ?person foaf:givenName ?givenName .
  ?person foaf:familyName ?familyName .
  ?person foaf:name ?fullName .
} WHERE {
  ?person vcard:given-name ?givenName .
  ?person vcard:family-name ?familyName .
  BIND(concat(?givenName," ",?familyName) AS ?fullName)
}

subject,predicate,object
http://www.snee.com/hr/emp1,http://xmlns.com/foaf/0.1/familyName,Smith
http://www.snee.com/hr/emp1,http://xmlns.com/foaf/0.1/givenName,Heidi
http://www.snee.com/hr/emp3,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://xmlns.com/foaf/0.1/Person
http://www.snee.com/hr/emp4,http://xmlns.com/foaf/0.1/familyName,Berger
http://www.snee.com/hr/emp3,http://xmlns.com/foaf/0.1/givenName,Francis
http://www.snee.com/hr/emp2,http://xmlns.com/foaf/0.1/givenName,John
http://www.snee.com/hr/emp1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://xmlns.com/foaf/0.1/Person
http://www.snee.com/hr/emp2,http://xmlns.com/foaf/0.1/name,John Smith
http://www.snee.com/hr/emp3,http://xmlns.com/foaf/0.1/familyName,Jones
http://www.snee.com/hr/emp2,http://xmlns.com/foaf/0.1/familyName,Smith
